### Panel app to show colormap artifacts 

#### Objective of the notebook
Building and deployment of a working app ... pain-free 

#### Do it yourself
Tutorial: [Panel: Dashboards for PyData](https://youtu.be/AXpjbJUVeb4), presented by James Bednar

Getting started guide: https://panel.holoviz.org/getting_started/index.html

#### License

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/"> CC BY Creative Commons License</a>, with the exception of the data used (a horizon surface from the [Penobscot 3D](https://terranubis.com/datainfo/Penobscot) which is covered by a [CC BY-SA Creative Commons License](https://creativecommons.org/licenses/by-sa/3.0/)).

#### Preliminaries

In [1]:
import os
import numpy as np
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.cm as cm
import colorcet
from skimage import io, color, exposure

In [2]:
from matplotlib import rc
font = {'size'   : 15}
rc('font', **font)

In [5]:
plt.switch_backend('agg')

In [7]:
import param
import panel as pn
pn.extension()

#### Helper functions

In [9]:
def normalise(data):
    """
    Normalize an array to [0-1] range
    """
    data_n = (data-np.amin(data)) / (np.amax(data)-np.amin(data))
    return data_n 

def Sobel_2d(data):
    """
    Calculate 2D Sobel edges
    """
    dx = ndi.sobel(data, 0)  # horizontal derivative
    dy = ndi.sobel(data, 1)  # vertical derivative
    mag = np.hypot(dx, dy)      # magnitude
    mag *= 255.0 / np.max(mag)  # normalize
    return mag 

In [12]:
def mk_cmapped_data(data, mpl_cmap_name):
    """
    This function makes a figure (but does not display it),
    plots the data with a colormap, then saves the 
    colormapped data from the canvas to a numpy array
     
    Updated after: https://stackoverflow.com/a/62518311/1034648
    """
    # Use `LinearSegementedColormap` and `Normalize` instances directly
    cmap = plt.get_cmap(mpl_cmap_name)
    norm = plt.Normalize(data.min(), data.max())

    # The norm instance scales data to a 0-1 range, cmap makes it RGB
    mat = cmap(norm(data))  

    # MPL uses a 0-1 float RGB representation, so scale to 0-255
    mat = (255 * mat).astype(np.uint8) 
    return mat

#### Make custom Blues colormap (with the darkest blue taken out) for the intensity plots

In [15]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    """
    function to truncate a matplotlib colormap
    from: https://stackoverflow.com/a/18926541/1034648
    """
    new_cmap = clr.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [17]:
cmap = plt.get_cmap('Blues_r')
Blues_r_t = truncate_colormap(cmap, 0.0, 0.8)

#### Load and register my colormaps 

For background check on my blog [here](https://mycarta.wordpress.com/color-palettes/).

With thanks to [@aadm](https://github.com/aadm): the logic for registering the colormaps in a loop is adapted from [this notebook of his](https://github.com/aadm/geophysical_notes/blob/master/colormaps.ipynb) !

In [20]:
data_folder = 'data'

In [24]:
names_and_files = {'matteo_cube': 'data/cube1_0-1.csv', 
                   'matteo_cubeYF': 'data/cubeYF_0-1.csv', 
                   'matteo_linear_L': 'data/Linear_L_0-1.csv'}

for n, f in names_and_files.items():
    tmp = clr.ListedColormap(np.loadtxt(f, delimiter=','), name=n)
    tmp_r = clr.ListedColormap(np.flipud(np.loadtxt(f, delimiter=',')), name=n+'_r')
    plt.colormaps.register(cmap=tmp, name=n)
    plt.colormaps.register(cmap=tmp_r, name=n+'_r')

#### Load the seismic horizon(from the [Penobscot 3D](https://terranubis.com/datainfo/Penobscot)) and normalise to [0 1] range

In [27]:
horiz_file = os.path.join(data_folder,'Penobscot_HorB.npy')
data = normalise(np.load(horiz_file)[55:-55,25:-25])

#### Text widget to explain (briefly at the moment) the purpose of the app

In [30]:
explain_text = 'A simple app to demonstrate the effect of colormaps on perception and on the ability \
to see fault traces as edges on a seismic horizon.'

In [32]:
explain_widget = pn.widgets.StaticText(name='Check perception effects', value= explain_text, width = 1200)

#### Text widget with instructions

In [35]:
select_text_1 = '(1) Look at the top row plots'
select_text_2 = '(2) select a cmap with the two drop-down menus'
select_text_3 = '(3) see the effect of cmap on bottom row plots'

In [37]:
text_widget = pn.Row(select_text_1, select_text_2, select_text_3, width = 1400)

In [39]:
text_widget

Row(width=1400)
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Markdown(str)

#### Function to show the reference plots in grayscale
Performs all the plotting tasks, including calls to the helper functions to create required variables

In [42]:
def make_original_plots(dt):
    """
    This function makes the reference plots, which will be the bottom ones in the app.
    It makes static displays of:
    1) On the left, the seismic data plotted with gray scale colormap
    2) In the middle, the gray scale intensity of the data
    3) On the right, the contrast-enhanced Sobel edges of the intensity
    """
    I = dt  # dt is already grayscale normalized data
    Sob = Sobel_2d(I) 
    fig = plt.figure(figsize = (24,6))   
    ax1 = fig.add_subplot(1, 3, 1)
    ax1.imshow(dt, cmap='gray', aspect = 'auto',  interpolation = 'none')
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title(' Horizon, grayscale')       
    ax3 = fig.add_subplot(1, 3, 2)
    ax3.imshow(I, cmap=Blues_r_t, aspect = 'auto', interpolation = 'none')
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.set_title('intensity')       
    ax5 = fig.add_subplot(1, 3, 3)
    ax5.imshow(Sob, cmap='gray_r', vmax = 70, aspect = 'auto', interpolation = 'none')
    ax5.set_xticks([])
    ax5.set_yticks([])
    ax5.set_title('Sobel edges')
    plt.tight_layout()                                    
    plt.close(fig=fig)   
    return fig

#### Class to make colormapped plots

This class links two parameters, one for the collection (either Matplotlib colormaps or Colorcet colormaps) and one for the colormap, so that when the collection is changed, only valid colormaps are allowed.

The link between the two parameters (collection and colormap) is achieved using the first `param.depends` decorator.

With the second `param.depends` decorator, when the colormap  is changed the plots are updated with the `make_cmapped_plots` method


In [46]:
class ColormappedPlots(param.Parameterized):
    
    collection = param.ObjectSelector(default='matplotlib', objects=['matplotlib', 'colorcet', 'mycarta'])
    
    colormap = param.ObjectSelector(default='cubehelix', objects=sorted([
    'viridis', 'plasma', 'inferno', 'magma', 'cividis',
    'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
    'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
    'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn',
    'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
    'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
    'hot', 'afmhot', 'gist_heat', 'copper', 
    'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
    'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic',
    'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
    'gnuplot', 'CMRmap', 'cubehelix', 'brg','gist_rainbow', 
    'rainbow', 'jet', 'nipy_spectral', 'gist_ncar'
    'viridis_r', 'plasma_r', 'inferno_r', 'magma_r', 'cividis_r',
    'Greys_r', 'Purples_r', 'Blues_r', 'Greens_r', 'Oranges_r', 'Reds_r',
    'YlOrBr_r', 'YlOrRd_r', 'OrRd_r', 'PuRd_r', 'RdPu_r', 'BuPu_r',
    'GnBu_r', 'PuBu_r', 'YlGnBu_r', 'PuBuGn_r', 'BuGn_r', 'YlGn_r',
    'binary_r', 'gist_yarg_r', 'gist_gray_r', 'gray_r', 'bone_r', 'pink_r',
    'spring_r', 'summer_r', 'autumn_r', 'winter_r', 'cool_r', 'Wistia_r',
    'hot_r', 'afmhot_r', 'gist_heat_r', 'copper_r', 
    'PiYG_r', 'PRGn_r', 'BrBG_r', 'PuOr_r', 'RdGy_r', 'RdBu_r',
    'RdYlBu_r', 'RdYlGn_r', 'Spectral_r', 'coolwarm_r', 'bwr_r', 'seismic_r',
    'flag_r', 'prism_r', 'ocean_r', 'gist_earth_r', 'terrain_r', 'gist_stern_r',
    'gnuplot_r', 'CMRmap_r', 'cubehelix_r', 'brg_r', 'gist_rainbow_r', 
    'rainbow_r', 'jet_r', 'nipy_spectral_r', 'gist_ncar_r'], key=str.casefold))
    
    _colormaps = {
        'matplotlib': sorted([
    'viridis', 'plasma', 'inferno', 'magma', 'cividis',
    'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
    'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
    'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn',
    'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
    'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
    'hot', 'afmhot', 'gist_heat', 'copper', 
    'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
    'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic',
    'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
    'gnuplot', 'CMRmap', 'cubehelix', 'brg','gist_rainbow', 
    'rainbow', 'jet', 'nipy_spectral', 'gist_ncar'
    'viridis_r', 'plasma_r', 'inferno_r', 'magma_r', 'cividis_r',
    'Greys_r', 'Purples_r', 'Blues_r', 'Greens_r', 'Oranges_r', 'Reds_r',
    'YlOrBr_r', 'YlOrRd_r', 'OrRd_r', 'PuRd_r', 'RdPu_r', 'BuPu_r',
    'GnBu_r', 'PuBu_r', 'YlGnBu_r', 'PuBuGn_r', 'BuGn_r', 'YlGn_r',
    'binary_r', 'gist_yarg_r', 'gist_gray_r', 'gray_r', 'bone_r', 'pink_r',
    'spring_r', 'summer_r', 'autumn_r', 'winter_r', 'cool_r', 'Wistia_r',
    'hot_r', 'afmhot_r', 'gist_heat_r', 'copper_r', 
    'PiYG_r', 'PRGn_r', 'BrBG_r', 'PuOr_r', 'RdGy_r', 'RdBu_r',
    'RdYlBu_r', 'RdYlGn_r', 'Spectral_r', 'coolwarm_r', 'bwr_r', 'seismic_r',
    'flag_r', 'prism_r', 'ocean_r', 'gist_earth_r', 'terrain_r', 'gist_stern_r',
    'gnuplot_r', 'CMRmap_r', 'cubehelix_r', 'brg_r', 'gist_rainbow_r', 
    'rainbow_r', 'jet_r', 'nipy_spectral_r', 'gist_ncar_r'], key=str.casefold),
                
        'colorcet': sorted([
    'cet_bgy', 'cet_bkr', 'cet_bgyw', 'cet_bky', 'cet_kbc', 'cet_coolwarm', 
    'cet_blues', 'cet_gwv', 'cet_bmw', 'cet_bjy', 'cet_bmy', 'cet_bwy', 'cet_kgy', 
    'cet_cwr', 'cet_gray', 'cet_dimgray', 'cet_fire', 'kb', 'cet_kg', 'cet_kr',
    'cet_colorwheel', 'cet_isolium', 'cet_rainbow', 'cet_bgy_r', 'cet_bkr_r', 
    'cet_bgyw_r', 'cet_bky_r', 'cet_kbc_r', 'cet_coolwarm_r', 'cet_blues_r', 
    'cet_gwv_r', 'cet_bmw_r', 'cet_bjy_r', 'cet_bmy_r', 'cet_bwy_r', 'cet_kgy_r', 
    'cet_cwr_r', 'cet_gray_r', 'cet_dimgray_r', 'cet_fire_r', 'kb_r', 'cet_kg_r', 
    'cet_kr_r', 'cet_colorwheel_r', 'cet_isolium_r', 'cet_rainbow_r']),
    
        'mycarta': sorted(['matteo_cube', 'matteo_cubeYF', 'matteo_linear_L'])
    }
    
    @param.depends('collection', watch=True)
    def _update_colormaps(self):
        colormaps = self._colormaps[self.collection]
        self.param['colormap'].objects = colormaps
        self.colormap = colormaps[0]

    @param.depends('colormap')     
    def make_cmapped_plots(self):
        """
        This function makes the reactive plots, the bottom ones in the app:
        1) On the left, the colormapped data plotted with the user selected colormap
        2) In the middle, the gray scale intensity of the colormapped data
        3) On the right, the contrast-enhanced Sobel edges of the intensity
        """
        mat = mk_cmapped_data(data, self.colormap)     # helper function to save 1st plot to RGB numpy array
        I_cmapped = color.rgb2gray(mat)   # helper function to convert the array to gray intensity
        Sob_cmapped = Sobel_2d(I_cmapped) # helper function to calculate 2D Sobel edges
        fig = plt.figure(figsize = (24,6))  
        ax2 = fig.add_subplot(1, 3, 1)
        ax2.imshow(mat, self.colormap, aspect = 'auto', interpolation = 'none')
        ax2.set_xticks([])
        ax2.set_yticks([])
        ax2.set_title('Colormapped horizon')  
        ax4 = fig.add_subplot(1, 3, 2)
        ax4.imshow(I_cmapped, cmap=Blues_r_t, aspect = 'auto', interpolation = 'none')
        ax4.set_xticks([])
        ax4.set_yticks([])
        ax4.set_title('Intensity')  
        ax6 = fig.add_subplot(1, 3, 3)
        ax6.imshow(Sob_cmapped, cmap='gray_r', vmax = 70, aspect = 'auto', interpolation = 'none')
        ax6.set_xticks([])
        ax6.set_yticks([])
        ax6.set_title('Sobel edges')
        plt.tight_layout()                                  
        plt.close(fig=fig)
        return fig

        
viewer = ColormappedPlots(name='Select Colormap below')

#### Compose the panels

In [49]:
ctool = pn.Column(
                    explain_widget,
                    text_widget, 
                    viewer.param,
                    make_original_plots(data),
                    viewer.make_cmapped_plots
                 )

ValueError: the input array must have size 3 along `channel_axis`, got (485, 413, 4)

#### Serve the app

In [20]:
ctool.servable(); # for server deployment

In [21]:
#ctool.show(); # for immediate deployment from notebook

__To do next:__

- ADD FABIO CRAMERI'S COLORMAP
- ADD CMOCEAN
- BAD COLORMAP EQUALIZATION TOOL
- RESPONSIVE SIZING